In [17]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action = 'ignore')
import os
import json
import re

In [18]:
DATA_DIR = 'C:/venvs/Programmers_DE/Final_Project/raw_data'

# 샘플 데이터 로딩
web = pd.read_csv(f'{DATA_DIR}/final_recipe_website.csv')
api = pd.read_json(f'{DATA_DIR}/api.json')
with open(f'{DATA_DIR}/video.json', 'rt', encoding = 'utf-8-sig') as json_file:
    video = json.load(json_file)

    
def json_to_csv(df):
    url = []
    thumbnails = []
    title = []
    description = []
    views = []
    uploaded_date = []
    likes = []
    comments = []
    first_comment = []
    for i in range(len(video)):
        url.append(video[i]['url'])
        thumbnails.append(video[i]['thumbnail'])
        title.append(video[i]['title'])
        description.append(video[i]['description'])
        views.append(video[i]['views'])
        uploaded_date.append(video[i]['uploaded_date'])
        likes.append(video[i]['likes'])
        comments.append(video[i]['uploaded_date'])
        first_comment.append(video[i]['first_comment'])
    
    data = {
        'recipe_link': url,
        'food_img':thumbnails,
        'food_name':title,
        'ingredient':description,
        'views':views,
        'created_date':uploaded_date,
        'likes':likes,
        'comments':comments,
        'first_comments':first_comment
    }
    return pd.DataFrame(data)

video = json_to_csv(video)

In [19]:
def website_preprocessing_metadata(df):
    ingredient_set = set()
    for i in range(len(df)):
        name_data = df['ingredient'][1].split('ingredient')[1].split('[')[i+2].split(']')[0]
        ingredient = name_data.replace('"', '').replace("'", '')
        ingredient_name = ingredient.split(',')
        for j in range(len(ingredient_name)):
            ingredient_set.add(ingredient_name[j])
    return ingredient_set

In [20]:
df = pd.read_csv('database_table.csv')

# 음식이름 - Meta Table로 분할

In [5]:
def make_meta_food(df):
    food_meta = df['food_name'].unique()
    food_meta_table = pd.DataFrame(food_meta, columns = ['food_name'])
    food_meta_table['food_id'] = range(100000, 100000 + len(food_meta_table))
    return food_meta_table

In [6]:
meta_food = make_meta_food(df)

In [7]:
meta_food

,food_name,food_id
0,초보도 만들기 쉬운 바스크 치즈케이크,100000
1,새우 단호박 찌개 만드는법,100001
2,햄버거 미니버거 모닝빵 햄버거 만들기 수제버거 패티 함박스테이크 만들기 대파크림치즈...,100002
3,수박 예쁘게 먹기좋게 자르기,100003
4,백종원 스팸짜글이 감자짜글이 고추장찌개 만들기,100004
...,...,...
1434,뚝배기 계란찜 l 백종원의 백종원 레시피,101434
1435,대용량 레시피 두 번째는 샐러드?사라다? 하던 대로 감자사라다 100인분 갈게유! ...,101435
1436,초간단 김치찌개 ㅣ 백종원의 백종원 레시피,101436
1437,"이것이 업소 스케일! 백종원의 대용량 레시피 첫 번째, 제육볶음 100인분 만들기 ...",101437


# 재료명 메타데이터로 분할

In [8]:
def make_meta_ingredient(df):
    ingredient_col = ['재료1', '재료2','재료3', '재료4', '재료5', '재료6', '재료7', '재료8', '재료9', '재료10']
    concat_series = pd.concat([df[col] for col in ingredient_col])
    unique_ingredient = concat_series.unique().tolist()
    ingredient_meta_table = pd.DataFrame(unique_ingredient, columns = ['ingredient_name'])
    ingredient_meta_table['ingredient_id'] = range(500000, 500000 + len(ingredient_meta_table))
    return ingredient_meta_table

In [9]:
meta_ingredient = make_meta_ingredient(df)

In [10]:
meta_ingredient['ingredient_name'].unique()[:30]

array(['크림치즈', '단호박', '수박', '고추장', '데리야끼소스', '뇨끼', '다진마늘', '소금', '감자',
       '새우두부계란찜연두부75g', '조선부추50g', '방울토마토소박이:방울토마토150g', '오이무침:오이70g',
       '북엇국북어채25g', '황태해장국황태15g', '된장국두부20g', '멸치육수:국물용멸치5g',
       '치커리샐러드:치커리30g', '스트로베리샐러드딸기70g', '그린매쉬드포테이토:감자80g',
       '버섯구이:새송이버섯70g', '구운채소:가지20g', '브로콜리컬리플라워샐러드:브로콜리40g',
       '오렌지당근펀치당근100g', '주재료:소면160g', '주재료실곤약440g', '주재료만두피4g',
       '주재료:소면50g육수:함초3g', '두부320g', '다시마1g'], dtype=object)

# 재료 정량 meta로 분할

In [11]:
def make_meta_ingredient(df):
    ingredient_unit_col = [ '재료1_정량',
       '재료2_정량', '재료3_정량', '재료4_정량', '재료5_정량', '재료6_정량', '재료7_정량', '재료8_정량',
       '재료9_정량', '재료10_정량']
    concat_series = pd.concat([df[col] for col in ingredient_unit_col])
    unique_ingredient = concat_series.unique().tolist()
    ingredient_unit_meta_table = pd.DataFrame(unique_ingredient, columns = ['ingredient_unit'])
    ingredient_unit_meta_table['ingredient_id'] = range(1500000, 1500000 + len(ingredient_unit_meta_table))
    return ingredient_unit_meta_table

In [12]:
meta_ingredient_unit = make_meta_ingredient(df)

In [25]:
meta_food.to_csv('meta_food.csv', index=False)
meta_ingredient.to_csv('meta_ingredient.csv', index=False)
meta_ingredient_unit.to_csv('meta_ingredient_unit.csv', index=False)

# 음식 카테고리 분할

In [25]:
DATA_DIR = 'C:/venvs/Programmers_DE/Final_Project/raw_data'
api = pd.read_json(f'{DATA_DIR}/api.json')

In [59]:
api_df = api_preprocessing_metadata(api)

In [ ]:
data

In [50]:
data_pre

[['새우두부계란찜연두부', '칵테일새우', '달걀', '생크림', '설탕', '무염버터'],
 ['조선부추', '날콩가루', '다진대파', '다진마늘', '고춧가루', '요리당', '참기름'],
 ['방울토마토소박이:방울토마토', '양파', '부추', '멸치액젓', '다진마늘2.', '매실액', '설탕', '물'],
 ['오이무침:오이', '다진땅콩', '사과'],
 ['북엇국북어채', '새우', '사과', '양파', '표고버섯', '물'],
 ['황태해장국황태', '콩나물', '무', '저염된장', '물', '청양고추', '다진마늘'],
 ['된장국두부', '애느타리버섯', '감자', '양파', '대파', '된장', '물'],
 ['멸치육수:국물용멸치', '다시마', '양파', '국간장', '물', '표고버섯', '다진마늘'],
 ['치커리샐러드:치커리', '적양배추', '양파', '당근', '식초', '설탕', '마늘'],
 ['스트로베리샐러드딸기', '플레인요거트', '양상추', '메추리알', '블루베리'],
 ['그린매쉬드포테이토:감자', '시금치우유소스', '아몬드', '설탕', '크랜베리', '치커리약간시금치우유소스:시금치', '우유'],
 ['버섯구이:새송이버섯', '올리브유', '다진양파', '다진오이피클', '올리브유', '식초', '레몬즙', '머스터드', '꿀'],
 ['구운채소:가지',
  '호박',
  '새송이버섯',
  '양파',
  '발사믹크레마',
  '빨강파프리카',
  '노랑파프리카',
  '청피망',
  '올리브유약간간장레몬소스:저염간장',
  '식초'],
 ['브로콜리컬리플라워샐러드:브로콜리',
  '컬리플라워',
  '적양파',
  '강낭콩',
  '건포도',
  '호두',
  '소금약간두유요거트소스:두유',
  '플레인요거트',
  '레몬즙'],
 ['오렌지당근펀치당근', '오렌지'],
 ['주재료:소면', '저염소금', '식초', '아몬드', '해바라기씨', '호두', '호박씨', '오이'],
 ['주재료실곤약', '

In [49]:
data_unit

[['75g', '20g', '30g', '13g', '5g', '5g'],
 ['50g', '7g', '5g', '2g', '2g', '2g', '2g'],
 ['150g', '10g', '10g', '3g', '5g', '2g', '2g', '2ml'],
 ['70g', '10g', '50g'],
 ['25g', '10g', '30g', '40g', '20g', '300ml'],
 ['15g', '30g', '30g', '10g', '300ml', '5g', '2g'],
 ['20g', '20g', '10g', '10g', '10g', '5g', '300ml'],
 ['5g', '1장', '10g', '5g', '300ml', '20g', '2g'],
 ['30g', '15g', '10g', '5g', '5g', '5g', '5g'],
 ['70g', '85g', '70g', '30g', '15g'],
 ['80g', '5g', '2g', '2g', '3g', '10g', '10g'],
 ['70g', '10g', '10g', '10g', '2g', '5g', '3g', '3g', '2g'],
 ['20g', '50g', '15g', '15g', '15g', '3g', '3g', '3g', '5g', '15g'],
 ['40g', '40g', '10g', '5g', '5알', '1/2알', '50g', '20g', '5g'],
 ['100g', '100g'],
 ['160g', '4g', '8g', '4g', '4g', '4g', '4g', '20g'],
 ['440g', '70g', '6g', '6g', '50g', '50g'],
 ['4g', '100g', '100g', '20g', '100g', '2g', '4g', '5g', '1g'],
 ['3g', '25g', '17g', '140g', '150g', '250g', '150g', '35g', '30g', '20g'],
 ['320g', '300g', '100g', '140g', '20g', '10

In [24]:
api.columns

Index(['API_ID', 'food_name', 'ingredient', 'serving', 'category',
       'category_2', 'food_img_1', 'food_img_2', 'recipe_01', 'recipe_02',
       'recipe_03', 'recipe_04', 'recipe_05', 'recipe_06', 'recipe_img_01',
       'recipe_img_02', 'recipe_img_03', 'recipe_img_04', 'recipe_img_05',
       'recipe_img_06', '재료1', '재료2', '재료3', '재료4', '재료5', '재료6', '재료7', '재료8',
       '재료9', '재료10', 'recipe_link'],
      dtype='object')

In [14]:
api['category'].unique()

array(['찌기', '기타', '끓이기', '굽기', '볶기', '튀기기'], dtype=object)

In [15]:
api['category_2'].unique()

array(['반찬', '국&찌개', '후식', '일품', '밥', '기타'], dtype=object)

In [73]:
df.columns

Index(['data_source', 'ID', 'food_name', 'food_img', 'recipe_link', 'serving',
       'category_high_level', 'category_mid_level', 'category_low_level',
       'created_date', 'views', 'likes', 'reviews', 'comments', '재료1', '재료2',
       '재료3', '재료4', '재료5', '재료6', '재료7', '재료8', '재료9', '재료10', '재료1_정량',
       '재료2_정량', '재료3_정량', '재료4_정량', '재료5_정량', '재료6_정량', '재료7_정량', '재료8_정량',
       '재료9_정량', '재료10_정량', 'rating'],
      dtype='object')

In [76]:
df[['food_name', '재료1', '재료2', '재료3', '재료4', '재료5', '재료6', '재료7', '재료8', '재료9', '재료10']]

,food_name,재료1,재료2,재료3,재료4,재료5,재료6,재료7,재료8,재료9,재료10
0,초보도 만들기 쉬운 바스크 치즈케이크,크림치즈,설탕,가는소금,달걀,생크림,박력분,NaN,NaN,NaN,NaN
1,새우 단호박 찌개 만드는법,단호박,칵테일새우,국간장,고추가루,다진마늘,다진 대파,물,NaN,NaN,NaN
2,햄버거 미니버거 모닝빵 햄버거 만들기 수제버거 패티 함박스테이크 만들기 대파크림치즈...,크림치즈,다진대파,꿀,바질,레몬즙,NaN,NaN,NaN,NaN,NaN
3,수박 예쁘게 먹기좋게 자르기,수박,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,백종원 스팸짜글이 감자짜글이 고추장찌개 만들기,고추장,고춧가루,된장,버섯가루,멸치가루,새우가루,다시마가루,들깨가루,맛술,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1441,뚝배기 계란찜 l 백종원의 백종원 레시피,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1442,대용량 레시피 두 번째는 샐러드?사라다? 하던 대로 감자사라다 100인분 갈게유! ...,오이,양파,당근 1.,셀러리,마요네즈 3.,황설탕,꽃소금,NaN,NaN,NaN
1443,초간단 김치찌개 ㅣ 백종원의 백종원 레시피,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1444,"이것이 업소 스케일! 백종원의 대용량 레시피 첫 번째, 제육볶음 100인분 만들기 ...",양파,양배추,돼지호박,청양고추,홍고추,풋고추,당근,대파,NaN,NaN


In [46]:
df.iloc[2]

data_source                                                      website
ID                                                               5000002
food_name              햄버거 미니버거 모닝빵 햄버거 만들기 수제버거 패티 함박스테이크 만들기 대파크림치즈...
food_img               https://recipe1.ezmember.co.kr/cache/recipe/20...
recipe_link                   https://www.10000recipe.com/recipe/7007908
serving                                                              3인분
category_high_level                                                  NaN
category_mid_level                                                   NaN
category_low_level                                                   NaN
created_date                                                  2023-08-11
views                                                                198
likes                                                                  0
reviews                                                                0
comments                                           